## Method1

In [1]:
import trimesh
import pyvista as pv
from utils import *

#### Load a file

In [2]:
mesh = pymesh.load_mesh("data/two_spheres2.ply")

In [3]:
self_intersection_stats(mesh)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |    1118 |
+-----------------------------------+---------+
| Number of faces                   |    1920 |
+-----------------------------------+---------+
| Number of intersecting face pairs |    2354 |
+-----------------------------------+---------+


#### Get outer hull (to remove inner components)

In [4]:
mesh = pymesh.compute_outer_hull(mesh)

In [5]:
self_intersection_stats(mesh)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |     842 |
+-----------------------------------+---------+
| Number of faces                   |    1680 |
+-----------------------------------+---------+
| Number of intersecting face pairs |       0 |
+-----------------------------------+---------+


In [6]:
visualize(mesh, filename="After compute_outer_hull()")

Widget(value='<iframe src="http://localhost:36823/index.html?ui=P_0x7f30f2eb7e50_0&reconnect=auto" class="pyvi…

None

#### Define method1


In [7]:
def method1(mesh, with_rounding=True, precision=11, max_iterations=15):
    """
    Resolves self-intersections in a mesh iteratively.

    Parameters:
        mesh: The input mesh object.
        with_rounding (bool): Enables rounding of vertices for stability. Default is True.
        precision (int): Rounding precision level for vertices.
        max_iterations (int): Maximum number of iterations allowed to resolve intersections.

    Returns:
        mesh: The processed mesh with no self-intersections.
    """
    # Initial rounding of vertices
    if (with_rounding):
        mesh = pymesh.form_mesh(
                np.round(mesh.vertices, precision),
                mesh.faces);
    intersecting_faces = pymesh.detect_self_intersection(mesh);

    # Iterative process to resolve self-intersections
    counter = 0;
    while len(intersecting_faces) > 0 and counter < max_iterations:
        if (with_rounding):
            involved_vertices = np.unique(mesh.faces[intersecting_faces].ravel());

            # Round only the involved vertices
            # Suppose precision = 4. Then,
            # [1.234567, 2.345678, 3.456789] <- One vertex example (x, y, z coords)
            # becomes
            # [1.23, 2.35, 3.46]
            vertices_copy = mesh.vertices.copy()  
            vertices_copy[involved_vertices, :] =\
                    np.round(mesh.vertices[involved_vertices, :],
                            precision//2);
        
            mesh = pymesh.form_mesh(vertices_copy, mesh.faces) 

        mesh = pymesh.resolve_self_intersection(mesh, "igl");
        mesh, __ = pymesh.remove_duplicated_faces(mesh, fins_only=True);
        if (with_rounding):
            mesh = pymesh.form_mesh(
                    np.round(mesh.vertices, precision),
                    mesh.faces);
        mesh = intermediate(mesh) # Reload mesh. Otherwise, the next step fails in some cases.
        intersecting_faces = pymesh.detect_self_intersection(mesh);
        print(len(intersecting_faces))
        counter += 1;

    if len(intersecting_faces) > 0:
        logging.warn("Resolving failed: max iteration reached!");

    return mesh

In [8]:
repaired_mesh = method1(mesh)

In [9]:
self_intersection_stats(repaired_mesh)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |     842 |
+-----------------------------------+---------+
| Number of faces                   |    1680 |
+-----------------------------------+---------+
| Number of intersecting face pairs |       0 |
+-----------------------------------+---------+


In [10]:
visualize(repaired_mesh, filename="After applying method1")

Widget(value='<iframe src="http://localhost:36823/index.html?ui=P_0x7f30b92e37f0_1&reconnect=auto" class="pyvi…

None